In [73]:
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sqlalchemy import create_engine

import pymysql

from tools import *

In [86]:
conn = pymysql.Connect(**MYSQL_MALL_DIC)
cursor = conn.cursor(cursor=pymysql.cursors.DictCursor)
engine_conn = "mysql+pymysql://{user}:{password}@{host}:{port}/{database}".format(**MYSQL_MALL_DIC)
engine = create_engine(engine_conn,encoding='utf-8')

In [87]:
order_df = pd.read_sql('select * from order_details where 下单时间>="2019-12-01 00:00:00"',engine)
goods_df = pd.read_sql('select * from goods',engine,)
# cursor.execute('select * from order_details where 下单时间>="2019-12-01 00:00:00"')
# d = cursor.fetchall()

In [88]:
free_order = order_df[order_df['goods_type']=="free"]

df = pd.merge(free_order,goods_df[["商品ID","SPUID","goods_name",'系统分类','市场价','成本价','售价','发货商','发货商ID','规格','单位']],on='商品ID',how='left')

In [90]:
df_r['lng'].dtype

dtype('float64')

In [92]:
df_r = df[['订单号','lng','lat']]
df_r = df_r.replace({"goods_name":{r".+?\((.+?)\)$":r"\1"}},regex=True)
l = []
for index, row in df_r.iterrows():
    l.append([row['lng'],row['lat'],1])

    
with open(r'D:\往期\echarts\data\heat_coord.json','w',encoding='utf-8') as f:
    f.write('points=')
    json.dump(l,f,ensure_ascii=False)

In [60]:
# 区域坐标
df_r = df[['订单号','国家','省份','城市','县区']]
df_r = df_r.replace({"goods_name":{r".+?\((.+?)\)$":r"\1"}},regex=True)
df_r  =df_r.groupby(['国家','省份','城市','县区']).count()
df_r = df_r.reset_index()
df_r['address'] = df_r['国家'] + df_r['省份']+df_r['城市'] + df_r['县区'] 
l  =[]
for index,row in df_r.iterrows():
    d = {
        "name":row['县区'],
    }

    loc = Geo()(row['address']).location

    d['value'] = [float(loc['lng']), float(loc['lat']), row['订单号']]
    l.append(d)
    print(loc)
with open(r'C:\Users\qhj01\Desktop\coord.json','w',encoding='utf-8') as f:
    f.write('coord_data=')
    json.dump(l,f,ensure_ascii=False)

In [6]:
df_r = df[['goods_name','订单号','国家','省份','城市','县区']]
df_r = df_r.replace({"goods_name":{r".+?\((.+?)\)$":r"\1"}},regex=True)

dt = pd.pivot_table(df_r,
         index=['国家','省份','城市','县区'],
         columns=['goods_name'],
         values=['订单号'],
         aggfunc={'订单号':"count"} ,fill_value=0,
               margins=True,
               margins_name="总计"
        )
dt = dt.stack(level=0).reset_index(level=-1,drop=True)
dt.columns.name = "三金护肤产品"
dt = dt[['水光针','补水','美肤','护肤','总计']].sort_index(level=1,ascending=False)

In [9]:
dt

三金护肤产品           水光针  补水  美肤  护肤   总计
国家 省份  城市  县区                        
中国 重庆市 重庆市 九龙坡区    0   0   1   0    1
   福建省 泉州市 晋江市     0   1   0   0    1
   湖南省 长沙市 雨花区     0   9   2   0   11
           长沙县     1   7   2   1   11
           芙蓉区     1  13   2   1   17
           望城区     0   1   0   0    1
           开福区     1  16   7   3   27
           岳麓区     2  10   2   3   17
           天心区     0   7   0   0    7
       邵阳市 北塔区     0   1   0   0    1
       衡阳市 石鼓区     0   1   0   0    1
       益阳市 赫山区     0   1   0   0    1
       湘潭市 湘乡市     0   2   0   0    2
       永州市 祁阳县     0   1   0   0    1
           冷水滩区    0   3   1   1    5
       株洲市 芦淞区     0   0   0   1    1
           天元区     0   1   1   0    2
       常德市 桃源县     0   2   0   0    2
   江苏省 无锡市 锡山区     0   0   1   0    1
   广东省 佛山市 南海区     0   0   1   0    1
       东莞市 东莞市     0   1   0   0    1
   天津市 天津市 河东区     0   1   0   0    1
总计                 5  78  20  10  113

In [7]:
# 折线图
df_r = df[['goods_name','下单时间','订单号']]
df_r = df_r.replace({"goods_name":{r".+?\((.+?)\)$":r"\1"}},regex=True)
df_r['下单日期'] = df_r['下单时间'].apply(lambda x:x.strftime('%Y-%m-%d'))
df_r.sort_values(by=['goods_name','下单时间'],inplace=True)

df_r = df_r[['goods_name','下单日期','订单号']]
df_r.rename(columns={"goods_name":"三金商品"},inplace=True)
df_r = df_r.groupby(['三金商品','下单日期']).count()
df_r = df_r.unstack(level=0).fillna(0)
df_r = df_r.stack(level=0).reset_index(level=-1,drop=True)

data = {}

data["data"] = []
data["xaxis"] = df_r.index.tolist()
for name, row in df_r[["补水","美肤","护肤","水光针"]].iteritems():
    d = {
        'name':name,
        'data':row.tolist()
    }
    data["data"].append(d)

js_path = r"C:\Users\qhj01\Desktop\test.json"
with open(js_path,"w",encoding='utf-8') as f:

    f.write('data=')

    json.dump(data,f,ensure_ascii=False)


In [8]:
df_r

三金商品,护肤,水光针,美肤,补水
下单日期,,,,
2019-12-12,4.0,1.0,5.0,32.0
2019-12-13,0.0,0.0,2.0,3.0
2019-12-14,0.0,0.0,3.0,8.0
2019-12-15,1.0,0.0,2.0,9.0
2019-12-16,2.0,1.0,0.0,4.0
2019-12-17,1.0,0.0,4.0,14.0
2019-12-18,0.0,0.0,4.0,7.0
2019-12-19,2.0,3.0,0.0,1.0


In [ ]:
stop
plt.rcParams['font.sans-serif'] = ['KaiTi'] # 指定默认字体
plt.rcParams['axes.unicode_minus'] = False


In [ ]:
df_r[['补水','护肤','护肤','水光针']].plot(kind='line',figsize=(16,6),rot = 45,
       grid = True)

In [ ]:
free_order.to_excel(r"C:\Users\qhj01\Desktop\TTTT.xlsx")